![header](header2.png)

# Handling exceptions, and debugging

## Exceptions

So far, errors have been the bane of our lives. We pass the wrong kind of argument to a function, we make an accidental division by 0, or we try to access a list element that doesn't exist. The whole program crashes, and we're frustrated trying to find out what went wrong.

Now we can turn this situation on its head. Python lets us anticipate places where errors might occur, and allows us to tell the program what to do if it encounters an error, instead of coming grinding to a halt. In other words, we can put our errors to work for us.

"Great!" you might be thinking. "I'll just ignore all errors!" Not so fast, of course. The point of error messages is they tell us where our program is going wrong and prevents more serious problems happening further down the line. We <i>want</i> to be informed when an unexpected error occurs as that means there's something wrong with our program. What exception handling allows us to do is manage the errors that we do expect to happen from time to time. This means our exception handling should be carefully tailored to the circumstances.

### <code>raise</code>

The <code>raise</code> keyword is followed by the name of an exception (a procedure that occurs when an error occurs), and causes that kind of exception to occur. We can also provide an error message to go with the error

In [1]:
raise TypeError("This is the wrong type of thing")

TypeError: This is the wrong type of thing

This can be used along with <code>if</code> statements to only raise the exception under specific circumstances. Suppose I have a function that is supposed to operate on lists:

In [2]:
def acts_on_a_list(a_list):
    for x in a_list:
        print(x)

Now before calling that function, I might write:

In [3]:
letters = ('S', 'a', 'm')
if not isinstance(letters, list):
    raise TypeError("letters variable should be a list")
acts_on_a_list(letters)

TypeError: letters variable should be a list

Now you see, I have anticipated that my function might receive the wrong kind of input at this stage, and given the user an error message to explain what they are doing wrong.

## <code>try</code>, <code>except</code>, <code>finally</code>

Astute reads will have noticed that actually, my function <code>acts_on_a_list()</code> doesn't require a list at all. It could take as its input anything sequential, such as a tuple, a dictionary, a range object, or maybe things we haven't even considered. I don't want to have to create <code>if</code>... <code>raise</code> statements for every possible kind of input my function could take, right?

This is where <code>try</code> comes in, and the mantra that "It's easier to ask forgiveness than permission". In short, <code>try</code> does what it says on the tin: it tries to do something! The difference is that we can provide further instructions in case what it is trying to do goes wrong. This would be more appropriate:

In [4]:
try:
    acts_on_a_list(letters)
except TypeError:
    print("acts_on_a_list was not provided with a sequence!")
    

S
a
m


In [5]:
try:
    acts_on_a_list(42)
except TypeError:
    print("acts_on_a_list was not provided with a sequence!")

acts_on_a_list was not provided with a sequence!


So this one didn't actually cause the error message. Instead, the exception was "caught" before the error message occurred, and a new bit of code was executed to say what happens next. This could be, for instance, performing an alternative version of the action:

In [6]:
try:
    acts_on_a_list(42)
except TypeError:
    print("acts_on_a_list was not provided with a sequence!")
    acts_on_a_list("Example")

acts_on_a_list was not provided with a sequence!
E
x
a
m
p
l
e


Notice, we have precisely specified what kind of error we wish to catch: other kinds of errors will still occur in the usual way.

In [7]:
try:
    acts_on_a_list(["HiPy", "Sam"][3])
except TypeError:
    print("acts_on_a_list was not provided with a sequence!")
    acts_on_a_list("Example")

IndexError: list index out of range

This is a good thing -- remember, we usually only want to catch the expected errors, and still be alerted properly if something truly unexpected happens. It's possible to use <code>pass</code> to continue as if nothing happened

In [8]:
try:
    acts_on_a_list(3.14)
except TypeError:
    pass
print("Look, nothing happened")

Look, nothing happened


And it seems like nothing happened. If we use <code>except</code> without providing the name of an exception, <i>all</i> exceptions will be handled in the same way (or ignored, if this is followed by <code>pass</code>). This is occasionally useful, but only if you know what you're doing -- specific, targeted errors are preferred. Here's a more practical little example of <code>try</code>, to add corresponding elements :

In [9]:
def add_list_elements(listA, listB):
    longest_list = max([listA, listB], key=len)
    listC = [x for x in longest_list]
    for i, value in enumerate(listC):
        try:
            listC[i] = listA[i] + listB[i]
        except IndexError:
            pass
    return listC

So we start by creating a copy of the longest list. Then for each index of the new list, we attempt to replace that list entry with the sum of the corresponding entries from the original of the two lists. If this fails due to an index error, which it eventually will if the lists are different sizes, we do nothing -- we just leave it as the same entry as the longer list:

In [10]:
A = [5, 5, 3, 123, 4]
B = [10, 20]
print(add_list_elements(A,B))

[15, 25, 3, 123, 4]


You can respond to different exceptions by giving them in sequence, as in:

In [ ]:
try:
    something()
except Exception1:
    do_this()
except Exception2:
    do_that()

<code>try</code>...<code>except</code> can be followed with an <code>else</code> clause, which occurs only if there was no exception.

A final part can be added after a <code>try</code>...<code>except</code> block. This is <code>finally</code>. This is a piece of code that will run regardless of success, failure, caught or uncaught exceptions. It will run no matter what, even if the rest of your program comes crashing down.

This is not unfamiliar -- it is precisely what <code>with open()</code> does when you open a file: it has a procedure for closing the file regardless of what happens while the file is open.

As an example of a good time to raise an exception, recall the polynomial class we made in tutorial 10. Virtually nothing will work if the argument provided is not a sequence of numbers, so we might want to add a check:

In [11]:
class Polynomial(list):
    def __init__(self, *coeffs):
        from collections.abc import Sequence
        from numbers import Number
        if isinstance(coeffs[0], Sequence):
            coeffs = coeffs[0]
        if not isinstance(coeffs, Sequence):
            raise TypeError("Argument should be a sequence of numbers")
        for x in coeffs:
            if not isinstance(x, Number):
                raise TypeError("Argument should be a sequence of numbers")
                
        coeffs = list(coeffs)
        list.__init__(coeffs)

In [12]:
Polynomial([3, 'a', 5])

TypeError: Argument should be a sequence of numbers

## Debugging

While we're on the subject of errors, we should talk a little bit about how to root out those bugs. One way that can be quite fruitful is simply heavy use of the <code>print()</code> function, telling Python to print various bits of informations as your program is running so you can spot where something has gone wrong. You can then "comment out" or delete the <code>print()</code> functions when you are done with them (and you probably should, since while printing might not seem like much it can slow a program down a surprising amount!).

Python provides a module that allows you to walk through your program step by step to see how it is working. Just import the module <code>pdb</code>, and you can get started.

Firstly, it's good not to have to debug your program from the start, but to choose a problematic part of the program to interrogate. Just add <code>pdb.set_trace()</code> before the bit of code you wish to examine.

Now when you run your code, you can interact with it as it is executing using simple commands. The first to know is that <code>q</code> is quit.

To advance through your program, you have two main options, <code>step</code> and <code>next</code>, which can be abbreviated as <code>s</code> or <code>n</code>. The difference is <code>s</code> will go "inside" a function on a line, but <code>n</code> will simply evaluate the function and move on to the next line. This distinction is not really clear from the names, so I like to imagine that <code>s</code> stands for "sub-routine", which is another word for "function" in computer programming. To skip ahead to the end of a function you are currently "inside", type <code>r</code>, which stands for return.

So, great, we can move through the lines of our program step by step. But what does this really tell us? Well, the great thing is, we can actually run any line of code or print any variable while we're inside. To run a line of code (for instance, to change a variable manually to see what happens), just type that line in, or prefix the line with a <code>!</code> if there is any risk of ambiguity. To print a variable inside the debugger, you can just type <code>p variable_name</code>, to save you typing <code>print()</code> every time. 

In [ ]:
### EXAMPLE VIDEO TO GO HERE